In [2]:
list = []

list.append(1)
list.append([1,2])
list.append(2)
list

[1, [1, 2], 2]

In [3]:
total_configurations = sum(200**n for n in range(1, 16))
print(total_configurations)


32932663316582914572864321608040200


In [ ]:
from google.colab import drive

drive.mount('mydrive')

Mounted at mydrive


In [ ]:
%pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import onnx

model = AutoModelForTokenClassification.from_pretrained('/content/mydrive/MyDrive/life/EXplore/мероприятия/Марафон "Цифровой Петербург"/FuzzySearch/Модели/mid_mybert_model')
model.eval()

tokenizer = AutoTokenizer.from_pretrained('/content/mydrive/MyDrive/life/EXplore/мероприятия/Марафон "Цифровой Петербург"/FuzzySearch/Модели/mid_mybert_model')




In [ ]:
import torch



dummy_input = tokenizer(
    "ул. Маяковская, 48, литера А",
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=128
)

torch.onnx.export(
    model,
    (dummy_input["input_ids"], dummy_input["attention_mask"]),
    "./onnxModel/midMybert.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {1: "seq_len"},
        "attention_mask": {1: "seq_len"},
        "logits": {1: "seq_len"}
    },
    opset_version=14  # можно и 11, но 14/15 лучше для новых моделей
)


In [ ]:
%pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
import onnxruntime
import numpy as np
from tokenizers import Tokenizer

# Загрузка токенизатора из файла (формат .json)
tokenizer = Tokenizer.from_file("/content/mydrive/MyDrive/life/EXplore/мероприятия/Марафон \"Цифровой Петербург\"/FuzzySearch/Модели/mid_mybert_model/tokenizer.json")




# Текст
text = "ул. Маяковская, 48, литера А"

# Токенизация
encoded = tokenizer.encode(text)

# Преобразование в нужный формат
input_ids = encoded.ids
attention_mask = encoded.attention_mask

# Приведение к фиксированной длине (например, 128)
max_length = 128
pad_id = tokenizer.token_to_id("[PAD]")  # Убедись, что у тебя есть [PAD] в словаре

input_ids += [pad_id] * (max_length - len(input_ids))
attention_mask += [0] * (max_length - len(attention_mask))


# Преобразуем в numpy массивы
input_ids = np.array([input_ids], dtype=np.int64)
attention_mask = np.array([attention_mask], dtype=np.int64)






# Загрузка модели
session = onnxruntime.InferenceSession("./onnxModel/midMybert.onnx")

# Получение выходов
outputs = session.run(
    None,
    {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }
)

# Результат
logits = outputs[0]
print(logits)

[[[ 9.284182   -0.29327655 -4.9524536  ... -5.7145615   1.679361
   -4.913538  ]
  [-0.13557161 11.479846   -3.0235097  ... -5.447598   -0.90085036
   -5.947455  ]
  [-1.0643938   8.643118   -4.32575    ... -2.7362385  -2.294745
   -2.4567583 ]
  ...
  [ 2.734609   -2.2718227  -3.8706071  ... -2.9900694  -2.5519392
   -2.212028  ]
  [-0.9710447  -1.9019011  -3.5006428  ...  5.2071257  -2.542507
   -1.5117273 ]
  [ 4.343253   -2.478183   -3.7869997  ... 15.2430725  -2.6414657
   -1.3749491 ]]]
